In [9]:
from bs4 import BeautifulSoup

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

driver = webdriver.Remote('http://localhost:4444/wd/hub', webdriver.DesiredCapabilities.CHROME)

# start the webdriver

/tmp/ipykernel_2829/714597795.py:7: DeprecationWarning: desired_capabilities has been deprecated, please pass in an Options object with options kwarg
  driver = webdriver.Remote('http://localhost:4444/wd/hub', webdriver.DesiredCapabilities.CHROME)


In [27]:
# get issues
driver.get("https://journals.sagepub.com/loi/JMX")

def get_url(x):
    soup = BeautifulSoup(x.get_attribute('innerHTML'), 'html.parser')
    return soup.a.get_attribute_list("href")[0]

issues = driver.find_elements(*(By.XPATH, "//div[@class='loi__issue__info']"))

[get_url(issue) for issue in issues]

[]

In [34]:
driver.save_screenshot("/home/ubuntu/ext_drive/HiWi/mlecon_scraping/ML-in-Business-Economics/screenshot.png")

True

In [33]:
driver.find_elements(*(By.XPATH, "//div[@class='loi__issue']"))

[]

In [18]:
test = driver.find_elements(*(By.XPATH, "//div[@class='loi__issue__info']"))

In [38]:
import os
os.environ['HF_DATASETS_CACHE'] = '/home/ubuntu/ext_drive/huggingface'

In [39]:
from datasets import load_dataset

dataset = load_dataset("s2orc", cache_dir="/home/ubuntu/ext_drive/huggingface")

KeyboardInterrupt: 